In [35]:
import tensorflow as tf
import os

try:
    # TPU 주소 가져오기
    tpu_address = os.environ['COLAB_TPU_ADDR']
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + tpu_address)
    print("TPU를 사용 중입니다:", tpu_address)
    
    # TPU 클러스터 연결 및 TPU 시스템 초기화
    tf.config.experimental_connect_to_cluster(resolver)
    tf.tpu.experimental.initialize_tpu_system(resolver)
    
    # TPU 전략 설정
    strategy = tf.distribute.TPUStrategy(resolver)
except KeyError:
    # TPU가 없을 경우 기본 전략 사용 (GPU/CPU)
    print("TPU가 발견되지 않아 GPU/CPU 전략을 사용합니다.")
    strategy = tf.distribute.get_strategy()  # GPU가 있으면 GPU, 없으면 CPU 사용

# 이후 `strategy`를 사용하여 모델 학습 진행 가능


TPU가 발견되지 않아 GPU/CPU 전략을 사용합니다.


In [36]:
# 딥러닝 모델 컴파일
def create_model():
  return tf.keras.Sequential(
      [
          tf.keras.layers.Conv2D(256, 3, activation='relu', input_shape=(28,28,1)),
          tf.keras.layers.Conv2D(256, 3, activation='relu'),
          tf.keras.layers.Flatten(),
          tf.keras.layers.Dense(256, activation='relu'),
          tf.keras.layers.Dense(128, activation='relu'),
          tf.keras.layers.Dense(10)
      ]
  )

with strategy.scope():
  model = create_model()
  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['sparse_categorical_accuracy'])

In [37]:
# 데이터 로드 및 정제
import pandas as pd
import numpy as np
import urllib.request
import os
from tqdm import tqdm
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel

In [38]:
# 데이터 로드
train_data = pd.read_csv('C:/Users/user/Desktop/AIML/datasets/train_set.csv', sep=',')
test_data = pd.read_csv('C:/Users/user/Desktop/AIML/datasets/test_set.csv', sep=',')

print('훈련용 리뷰 개수 :',len(train_data)) # 훈련용 리뷰 개수 출력
print('테스트용 리뷰 개수 :',len(test_data)) # 테스트용 리뷰 개수 출력

훈련용 리뷰 개수 : 80713
테스트용 리뷰 개수 : 20179


In [39]:
# BERT 모델 생성
def convert_examples_to_features(sentences, labels, max_seq_len, tokenizer):
  input_ids, attention_masks, token_type_ids, data_labels = [], [], [], []

  for example, label in tqdm(zip(sentences, labels), total=len(sentences)):
    # input_id는 워드 임베딩을 위한 문장의 정수 인코딩
    input_id = tokenizer.encode(example, max_length=max_seq_len, pad_to_max_length=True)

    # attention_mask 설정 - 실제 단어 존재시 1, 패딩이면 0
    padding_count = input_id.count(tokenizer.pad_token_id)
    attention_mask = [1] * (max_seq_len - padding_count) + [0] * padding_count

    # token_type_id는 문장을 구분하기 위한 인덱스
    token_type_id = [0] * max_seq_len

    assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
    assert len(attention_mask) == max_seq_len, "Error with attention mask length {} vs {}".format(len(attention_mask), max_seq_len)
    assert len(token_type_id) == max_seq_len, "Error with token type length {} vs {}".format(len(token_type_id), max_seq_len)

    input_ids.append(input_id)
    attention_masks.append(attention_mask)
    token_type_ids.append(token_type_id)
    data_labels.append(label)

  # np.array로 변환
  input_ids = np.array(input_ids, dtype=int)
  attention_masks = np.array(attention_masks, dtype=int)
  token_type_ids = np.array(token_type_ids, dtype=int)
  data_labels = np.asarray(data_labels, dtype=np.int32)

  return (input_ids, attention_masks, token_type_ids), data_labels

In [40]:
# BERT Tokenizer 설정
max_seq_len = 128
tokenizer = BertTokenizer.from_pretrained('klue/bert-base')

# 훈련 데이터를 토큰화하여 BERT 모델에 맞게 변환
train_encodings = tokenizer(
    train_data['data'].tolist(),
    max_length=max_seq_len,
    truncation=True,
    padding='max_length',
    return_tensors='tf'
)

# 레이블을 Tensor 형식으로 변환
train_labels = tf.convert_to_tensor(train_data['spam'].tolist())

# 최종 훈련 데이터셋 (train_X, train_y)
train_X = train_encodings
train_y = train_labels


In [41]:
# BERT Tokenizer 설정
max_seq_len = 128
tokenizer = BertTokenizer.from_pretrained('klue/bert-base')

# 테스트 데이터를 토큰화하여 BERT 모델에 맞게 변환
test_encodings = tokenizer(
    test_data['data'].tolist(),  # test_data의 'data' 열 사용
    max_length=max_seq_len,
    truncation=True,         # 시퀀스가 max_length를 넘을 경우 자르기
    padding='max_length',    # 시퀀스를 max_length에 맞춰 패딩
    return_tensors='tf'
)

# 레이블을 Tensor 형식으로 변환
test_labels = tf.convert_to_tensor(test_data['spam'].tolist())  # test_data의 'spam' 열 사용

# 최종 테스트 데이터셋 (test_X, test_y)
test_X = test_encodings
test_y = test_labels


In [46]:
# 훈련 데이터의 첫 번째 샘플 출력
input_id = train_X['input_ids'][0]       # 'input_ids' 키를 사용
attention_mask = train_X['attention_mask'][0]  # 'attention_mask' 키를 사용
token_type_id = train_X['token_type_ids'][0]   # 'token_type_ids' 키를 사용 (필요시 사용)

label = train_y[0]

print('단어에 대한 정수 인코딩 :', input_id)
print('어텐션 마스크 :', attention_mask)
print('세그먼트 인코딩 :', token_type_id)
print('각 인코딩의 길이 :', len(input_id))
print('정수 인코딩 복원 :', tokenizer.decode(input_id))
print('출력 샘플의 레이블 :', label)


단어에 대한 정수 인코딩 : tf.Tensor(
[    2 11782  2311  2250     1  4648  1868  2477  2121  2372  2062  2076
  8330  2010  6609    85    61  2309  2611  2008  2659  2189  2088  2272
    55  2041  2296     1 22316  2046  2046   645  2747  4930 14995  7464
  4589     3     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0], shape=(128,), dtype=int32)
어텐션 마스크 : tf.Tensor(
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [51]:
from tensorflow.keras.layers import Dropout
from tensorflow.keras import regularizers

# BERT 모델 정의 (모델 복잡도 조정: 드롭아웃 비율 조정 및 L2 정규화 적용)
class TFBertForSequenceClassification(tf.keras.Model):
    def __init__(self, model_name, l2_reg=0.01, dropout_rate=0.5):  # dropout_rate를 0.5로 증가
        super(TFBertForSequenceClassification, self).__init__()
        self.bert = TFBertModel.from_pretrained(model_name, from_pt=True)
        self.dropout = Dropout(dropout_rate)  # 드롭아웃 레이어 추가
        self.classifier = tf.keras.layers.Dense(
            1,
            kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.02),
            activation='sigmoid',
            kernel_regularizer=regularizers.l2(l2_reg),  # L2 정규화 적용
            name='classifier'
        )

    def call(self, inputs):
        input_ids, attention_mask, token_type_ids = inputs
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        cls_token = outputs[1]
        cls_token = self.dropout(cls_token)  # 드롭아웃 적용
        prediction = self.classifier(cls_token)
        return prediction


In [52]:
from transformers import TFBertForSequenceClassification
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping

# Early Stopping 정의
early_stopping = EarlyStopping(monitor='val_loss', patience=1, restore_best_weights=True)

# 훈련 데이터셋 생성
dataset = tf.data.Dataset.from_tensor_slices(({
    'input_ids': tf.convert_to_tensor(train_X['input_ids']),
    'attention_mask': tf.convert_to_tensor(train_X['attention_mask']),
    'token_type_ids': tf.convert_to_tensor(train_X['token_type_ids'])
}, train_y)).shuffle(buffer_size=len(train_y))

# 데이터셋 분할 (80% train, 20% validation)
train_size = int(0.8 * len(train_y))
train_dataset = dataset.take(train_size).batch(32)
val_dataset = dataset.skip(train_size).batch(32)

# TPU 또는 GPU/CPU 전략 설정
try:
    tpu_address = os.environ['COLAB_TPU_ADDR']
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + tpu_address)
    tf.config.experimental_connect_to_cluster(resolver)
    tf.tpu.experimental.initialize_tpu_system(resolver)
    strategy = tf.distribute.TPUStrategy(resolver)
except KeyError:
    strategy = tf.distribute.get_strategy()

# 모델 학습 및 컴파일
with strategy.scope():
    model = TFBertForSequenceClassification.from_pretrained('klue/bert-base', from_pt=True)
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
    loss = tf.keras.losses.BinaryCrossentropy()
    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

# 모델 학습
model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=3,
    callbacks=[early_stopping]
)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.encoder.layer.1.attention.output.LayerNorm.bias', 'bert.encoder.layer.10.attention.output.LayerNorm.bias', 'bert.encoder.layer.2.output.dense.weight', 'bert.encoder.layer.6.attention.output.LayerNorm.weight', 'bert.encoder.layer.10.attention.self.value.bias', 'bert.encoder.layer.0.attention.output.LayerNorm.bias', 'bert.encoder.layer.9.attention.self.value.weight', 'bert.encoder.layer.10.intermediate.dense.weight', 'bert.encoder.layer.0.intermediate.dense.weight', 'bert.encoder.layer.1.attention.self.value.bias', 'bert.encoder.layer.7.output.LayerNorm.weight', 'bert.encoder.layer.0.attention.self.key.weight', 'bert.encoder.layer.3.output.LayerNorm.weight', 'bert.encoder.layer.5.output.dense.bias', 'bert.encoder.layer.5.attention.output.LayerNorm.bias', 'bert.encoder.layer.0.attention.output.dense.weight', 'bert.encoder.layer.7.output.LayerNorm.bias', 'bert.encoder.

Epoch 1/3
2018/2018 [==============================] - 1429s 702ms/step - loss: 0.0276 - accuracy: 0.6767 - val_loss: 0.0020 - val_accuracy: 0.9900
Epoch 2/3
2018/2018 [==============================] - 1385s 686ms/step - loss: 4.3764 - accuracy: 0.6564 - val_loss: 6.0570 - val_accuracy: 0.6028


In [53]:
# 테스트 데이터셋의 'input_ids', 'attention_mask', 'token_type_ids'로 나누어 평가 입력으로 사용
test_input_ids = test_X['input_ids']
test_attention_mask = test_X['attention_mask']
test_token_type_ids = test_X['token_type_ids']

# 모델 평가
results = model.evaluate(
    (test_input_ids, test_attention_mask, test_token_type_ids),
    test_y,
    batch_size=1024
)
print("test loss, test acc: ", results)

20/20 [==============================] - 118s 6s/step - loss: 0.0017 - accuracy: 0.9908
test loss, test acc:  [0.001748122158460319, 0.9908320307731628]


In [ ]:
# 리뷰 예측해보기
def sentiment_predict(new_sentence):
    input_id = tokenizer.encode(new_sentence, max_length=max_seq_len, pad_to_max_length=True)
    padding_count = input_id.count(tokenizer.pad_token_id)
    attention_mask = [1] * (max_seq_len - padding_count) + [0] * padding_count
    token_type_id = [0] * max_seq_len

    input_ids = np.array([input_id])
    attention_masks = np.array([attention_mask])
    token_type_ids = np.array([token_type_id])

    encoded_input = [input_ids, attention_masks, token_type_ids]
    score = model.predict(encoded_input)[0]

    # score의 첫 번째 요소만 사용하여 스칼라 값으로 변환
    score = score[0] if isinstance(score, (np.ndarray, tf.Tensor)) else score

    if score > 0.5:
        print("{:.2f}% 확률로 햄 메세지입니다.\n".format(score * 100))
    else:
        print("{:.2f}% 확률로 스팸 메세지입니다.\n".format((1 - score) * 100))

In [55]:
# Model test
sentiment_predict("안녕")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
d:\Anaconda3\envs\AIML\lib\site-packages\transformers\tokenization_utils_base.py:2383: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


1/1 [==============================] - 2s 2s/step


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
# 모델 저장하기

# Save the entire model to a directory
# model.save('/content/drive/MyDrive/AIML/Codes/NLP/Modeling/KoBERT_Practice_Model')